# Text as Data

## Libraries

In [7]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np

# DNP Contexto
from contexto.limpieza import *  # Custom cleaning methods (assumes a specific library)

# Text preprocessing and categorical encoding
from sklearn.feature_extraction.text import CountVectorizer  # Bag of Words model
from sklearn.preprocessing import LabelEncoder  # Encoding categorical variables

# Natural Language Processing
import spacy  # Lemmatization

# Dimensionality reduction
from sklearn.decomposition import PCA  # Principal Component Analysis
from sklearn.preprocessing import StandardScaler  # Standardizing features

# Model evaluation and training
import xgboost as xgb # XGBoost model
from sklearn.model_selection import train_test_split  # Splitting datasets
from sklearn.linear_model import LinearRegression, ElasticNet  # Regression models
from sklearn.metrics import mean_absolute_error  # Performance evaluation metric

# Graphs
import matplotlib.pyplot as plt

## Data Loading

In [8]:
# Directory
os.chdir("/Users/sergiosandovalcamargo/Desktop/Problem Set 3")  

# Train and test data
train = pd.read_csv("stores/data/raw/train_test/train.csv") 
test = pd.read_csv("stores/data/raw/train_test/test.csv") 

In [1]:
# Combined data 
train['is_test'] = 0  
test['is_test'] = 1   
data = pd.concat([train, test], ignore_index=True)

NameError: name 'train' is not defined

## Clean Text

### Normalization, Stop Words and Lemmatization

In [10]:
# Spanish stop words: 608
with open('stores/data/raw/external/spanish.txt', 'r', encoding='utf-8') as file:
    stopwords = {line.strip() for line in file}

# Additional Stop words 
additional_stopwords = ['vendo', 'venta', 'vende', 'etc', 'carrera', 'calle', 'casa', 'apto', 'apartamento',
                        'propiedad', 'venta', 'inmueble', 'cuarto', 'habitacion', 'excelente', 'ubicado', 'area',
                        'espectacular', 'magnifico', 'muy', 'vivienda', 'piso', 'alcoba', 'bano', 'bao', 'via', 'mas',
                        'consta', 'bogota', 'santa', 'mts', 'metro']

stopwords.update(additional_stopwords)

In [11]:
# Spacy model
nlp = spacy.load('es_core_news_sm', disable=['parser', 'ner'])

# Clean text function
def clean_text(text):
    if pd.isnull(text):  # Manejo de valores nulos
        return ""
    else:
        # First cleaning filter
        text_clean = limpieza_texto(text, quitar_acentos=True, n_min=3, lista_palabras=stopwords)
        
        # Lemmatization
        doc = nlp(text_clean)
        text_clean = ' '.join(token.lemma_ for token in doc)

        # Remove plural 's
        text_clean = re.sub(r'\b(\w+)s\b', r'\1', text_clean)
        
        # Second cleaning filter
        text_clean = limpieza_texto(text_clean, quitar_acentos=True, n_min=3, lista_palabras=stopwords)
        
        return text_clean

In [12]:
# Apply function
data['cleaned_text'] = data['description'].apply(clean_text)

### Bag of Words

In [13]:
# Dummies variables from bag of words
def dummies_variables_sparse(data, min_freq, ngram_range=(1, 2)):

    # Bag of Words
    vectorizer = CountVectorizer(binary=True, ngram_range=ngram_range)
    X_bow = vectorizer.fit_transform(data['cleaned_text'])
    
    # Sparse DataFrame
    bow_df = pd.DataFrame.sparse.from_spmatrix(X_bow, columns=vectorizer.get_feature_names_out(), index=data.index)
    
    # Minimum frequency
    word_freq = bow_df.mean(axis=0)
    frequent_words = word_freq[word_freq >= min_freq].index
    bow_df_filtered = bow_df[frequent_words]
    
    # Dummy varuables
    bow_df_filtered = bow_df_filtered.add_prefix('d_')
    data_with_dummies = pd.concat([data, bow_df_filtered], axis=1)
    return data_with_dummies

In [14]:
# Apply function
data_large = dummies_variables_sparse(data, min_freq=0.01)
data_med = dummies_variables_sparse(data, min_freq=0.05)
data_light = dummies_variables_sparse(data, min_freq=0.1)

In [15]:
# Size
print(data_large.shape[1])
print(data_med.shape[1])
print(data_light.shape[1])

704
148
87


In [16]:
# Combined repeated words function
def combine_dummy_variables(data, variable_pairs):
    copy = data.copy()
    for var1, var2 in variable_pairs:
        new_var = var1.split()[0][:-1]
        copy[new_var] = copy[[var1, var2]].max(axis=1)
        copy.drop(columns=[var1, var2], inplace=True)
    return copy

# Repeated variables
variable_pairs = [
    ('d_balcn', 'd_balcon'),
    ('d_chimenea', 'd_chimeneo'),
    ('d_vista', 'd_visto'),
    ('d_cocin', 'd_cocina'),
    ('d_comedor cocin', 'd_comedor cocina'),
    ('d_iluminacion', 'd_iluminado'),
    ('d_saln', 'd_salon'),
    ('d_vigilancia hora', 'd_hora')]

In [17]:
# Apply function
data_med_clean = combine_dummy_variables(data_med, variable_pairs)